## 2026 EY AI & Data Challenge - Landsat Data Extraction Notebook

This notebook demonstrates Landsat data extraction and the creation of an output file to be used by the benchmark notebook. The baseline data is [Landsat Collection 2 Level 2](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) data from the MS Planetary Computer catalog. 

<b>Caution</b> ... This notebook requires significant execution time as there are 9319 data points (unique locations and times) used for data extraction from the Landsat archive. The code takes about 7 hours to run to completion on a typical laptop computer and typical internet connection. Lower execution times are likely possible with optimization of the data extraction process and use of cloud computing services. 

### Load Python Dependencies

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os

<h3>Extracting Landsat Data Using API Calls</h3> <p align="justify"> The API-based method allows us to efficiently access <b>Landsat</b> data for specific coordinates and time periods, ensuring scalability and reproducibility of the process. </p> <p align="justify"> Through the API, we can query individual bands or compute indices like <b>NDMI</b> on-the-fly. This approach reduces storage requirements and simplifies data preprocessing, making it ideal for large-scale environmental and water quality analysis. </p>

<p>The <b>compute_Landsat_values</b> function extracts Landsat surface reflectance values for specific sampling locations using a 100 m focal buffer around each point. For each location:</p>

<ul>
  <li>A bounding box (bbox) is created around the latitude and longitude coordinates.</li>
  <li>The Microsoft Planetary Computer API is queried for Landsat-8 Level-2 surface reflectance imagery within the date range.</li>
  <li>The nearest low-cloud (<10% cloud cover) scene is selected, and the specified bands (<b>green</b>, <b>nir08</b>, <b>swir16</b>, <b>swir22</b>) are loaded.</li>
  <li>Median values of the pixels within the bounding box are computed to reduce the effect of noise or outliers.</li>
</ul>

<p><b>Why the buffer value is 0.00089831:</b></p>

<p>We want a ~100 m buffer around each point. At the equator, 1 degree ≈ 110 km. Therefore, the degree equivalent of 100 m is:</p>

<p style="text-align:center;">
  <em>buffer_deg = 100 m / 110,000 m/deg ≈ 0.00089831</em>
</p>

<p>This slightly adjusted value ensures that the buffer approximately matches the pixel resolution of Landsat imagery, capturing a ~100 m area around each sampling location.</p>


In [2]:
# Setup
tqdm.pandas()

def compute_Landsat_values(row):
    lat = row['Latitude']
    lon = row['Longitude']
    date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')

    # Buffer size for ~100m 
    bbox_size = 0.00089831  
    bbox = [
        lon - bbox_size / 2,
        lat - bbox_size / 2,
        lon + bbox_size / 2,
        lat + bbox_size / 2
    ]

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    # Wider search range, we'll filter to nearest date later
    search = catalog.search(
        collections=["landsat-c2-l2"],
        bbox=bbox,
        datetime="2011-01-01/2015-12-31",
        query={"eo:cloud_cover": {"lt": 10}},
    )
    
    items = search.item_collection()

    if not items:
        return pd.Series({
            "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
        })

    try:
        # Convert sample date to UTC
        sample_date_utc = date.tz_localize("UTC") if date.tzinfo is None else date.tz_convert("UTC")

        # Pick the item closest to the sample date
        items = sorted(
            items,
            key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc)
        )
        selected_item = pc.sign(items[0])

        # Load required bands
        bands_of_interest = ["green", "nir08", "swir16", "swir22"]
        data = stac_load([selected_item], bands=bands_of_interest, bbox=bbox).isel(time=0)

        green = data["green"].astype("float")
        nir = data["nir08"].astype("float")
        swir16 = data["swir16"].astype("float")
        swir22 = data["swir22"].astype("float")
        
        # Compute medians
        median_green = float(green.median(skipna=True).values)
        median_nir = float(nir.median(skipna=True).values)
        median_swir16 = float(swir16.median(skipna=True).values)
        median_swir22 = float(swir22.median(skipna=True).values)

        # Replace 0 with NaN
        median_green = median_green if median_green != 0 else np.nan
        median_nir = median_nir if median_nir != 0 else np.nan
        median_swir16 = median_swir16 if median_swir16 != 0 else np.nan
        median_swir22 = median_swir22 if median_swir22 != 0 else np.nan
        
        return pd.Series({
            "nir": median_nir,
            "green": median_green,
            "swir16": median_swir16,
            "swir22": median_swir22,
        })
    
    except Exception as e:
        return pd.Series({
            "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
        })


### Extracting features for the training dataset

In [3]:
Water_Quality_df=pd.read_csv('water_quality_training_dataset.csv')
Water_Quality_df.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [4]:
Water_Quality_df.shape

(9319, 6)

In [5]:
Water_Quality_df_200 = Water_Quality_df.loc[0:199]
Water_Quality_df_200.shape

(200, 6)

<h3>Note:</h3>
<p>The Landsat data extraction process for all 9,319 locations typically requires 7+ hours when executed in a single run. During long executions, you may occasionally encounter API limits, timeout errors, or request failures. To avoid these interruptions, we recommend running the extraction in smaller batches.</p>

<p>In this notebook, we provide a sample code snippet demonstrating how to extract data for the first 200 locations. Participants are encouraged to follow the same batching approach to extract data for all 9,319 locations safely and efficiently.</p>

<p>We have already executed the full extraction for all 9,319 locations and saved the output to <b>landsat_features_training.csv</b>, which will be used in the benchmark notebook.
Similarly, participants can extract Landsat features in batches, combine the batch outputs, and save the final merged dataset as <b>landsat_features_training.csv</b> to ensure the benchmark notebook runs smoothly.</p>

In [6]:
# Extract band values from Landsat for training dataset
train_features_path = "landsat_features_training_200.csv"

print("🚀 Running Landsat feature extraction for training data...")
landsat_train_features = Water_Quality_df_200.progress_apply(compute_Landsat_values, axis=1)
landsat_train_features.to_csv(train_features_path, index=False)

🚀 Running Landsat feature extraction for training data...


100%|██████████| 200/200 [08:15<00:00,  2.48s/it]


<p><b>NDMI and MNDWI Indices:</b></p>
<p>In this notebook, we compute two commonly used water-related indices from the extracted Landsat bands:</p>
<ul>
  <li><b>NDMI (Normalized Difference Moisture Index):</b> Measures vegetation water content and surface moisture. Computed as <em>(NIR - SWIR16) / (NIR + SWIR16)</em>.</li>
  <li><b>MNDWI (Modified Normalized Difference Water Index):</b> Highlights open water features by enhancing water reflectance and suppressing built-up areas. Computed as <em>(Green - SWIR16) / (Green + SWIR16)</em>.</li>
</ul>

<p>An <b>epsilon value</b> (<em>eps = 1e-10</em>) is added in the denominators to avoid division by zero. These indices are widely used in hydrological and water quality analyses for detecting water presence and vegetation moisture levels.</p>


In [7]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_train_features['NDMI'] = (landsat_train_features['nir'] - landsat_train_features['swir16']) / (landsat_train_features['nir'] + landsat_train_features['swir16'] + eps)
landsat_train_features['MNDWI'] = (landsat_train_features['green'] - landsat_train_features['swir16']) / (landsat_train_features['green'] + landsat_train_features['swir16'] + eps)

In [8]:
landsat_train_features['Latitude'] = Water_Quality_df['Latitude']
landsat_train_features['Longitude'] = Water_Quality_df['Longitude']
landsat_train_features['Sample Date'] = Water_Quality_df['Sample Date']
landsat_train_features = landsat_train_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [9]:
landsat_train_features.to_csv(train_features_path, index=False)

In [10]:
# Preview File
landsat_train_features.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683


### Extracting features for the validation dataset

In [11]:
Validation_df=pd.read_csv('submission_template.csv')
Validation_df.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN


In [12]:
Validation_df.shape

(200, 6)

In [13]:
# Extract band values from Landsat for submission dataset
val_features_path = "landsat_features_validation.csv"

print("🚀 Running Landsat feature extraction for validation data...")
landsat_val_features = Validation_df.progress_apply(compute_Landsat_values, axis=1)
landsat_val_features.to_csv(val_features_path, index=False)

🚀 Running Landsat feature extraction for validation data...


100%|██████████| 200/200 [10:38<00:00,  3.19s/it]


In [14]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_val_features['NDMI'] = (landsat_val_features['nir'] - landsat_val_features['swir16']) / (landsat_val_features['nir'] + landsat_val_features['swir16'])
landsat_val_features['MNDWI'] = (landsat_val_features['green'] - landsat_val_features['swir16']) / (landsat_val_features['green'] + landsat_val_features['swir16'] + eps)

In [15]:
landsat_val_features['Latitude'] = Validation_df['Latitude']
landsat_val_features['Longitude'] = Validation_df['Longitude']
landsat_val_features['Sample Date'] = Validation_df['Sample Date']
landsat_val_features = landsat_val_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [16]:
landsat_val_features.to_csv(val_features_path, index=False)

In [17]:
# Preview File
landsat_val_features.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI
0,-32.043333,27.822778,01-09-2014,15229.0,12868.0,14797.0,12421.0,0.014388,-0.069727
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,16221.0,9304.5,12536.5,9958.0,0.128123,-0.147979
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,9125.0,11100.5,9455.0,8711.0,-0.017761,0.080052
